## IMPORTS and UTILS

In [1]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np
import json
import csv



import matplotlib.pyplot as plt


import os

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC
from robust_evaluation_tools.robust_utils import get_site, robust_text, rwp_text
from robust_evaluation_tools.robust_harmonization import fit, apply, visualize_harmonization, QC, compare_with_compilation, create_presentation, compare_distances


CAMCAN = "./DONNES/CamCAN.md.raw.csv.gz"
COMPILATION = "./DONNES/adni_compilation.csv.gz"

SYNTHETIC_SITES = "ROBUST/SYNTHETIC_SITES"

MAINFOLDER = "ROBUST"

RAWFOLDER = "RAW"

ANALYSISFOLDER = "ANALYSIS"

hi


## HARMONIZATION

In [18]:
def harmonize(f_train, f_test, directory, robust, rwp,hc):
    os.makedirs(directory, exist_ok=True)
    print(f_train)
    
    # Fit the model
    output_model_filename = fit(f_train, robust, rwp, directory, hc)
    output_model_filename = os.path.join(directory, output_model_filename)
    # Apply the model
    output_filename = apply(f_test, output_model_filename, robust, rwp, directory) 
    
    # Perform quality control
    dists, bundle_names = QC(output_filename, output_model_filename)
    dists_df = pd.DataFrame([dists], columns=bundle_names)
    dists_df['site'] = get_site(f_train)
    
    # Visualize the harmonization
    visualize_harmonization(f_test, output_filename, directory)

    mea = compare_with_compilation(pd.read_csv(output_filename))
    mea['site'] = get_site(f_train)
    
    # If robust is not "No", load metrics and outliers
    if robust != "No":
        metrics_filename = os.path.join(directory, f"metrics_{get_site(f_train)}_{robust_text(robust)}_{rwp_text(rwp)}.csv")
        outliers_filename = os.path.join(directory, f"outliers_{get_site(f_train)}_{robust_text(robust)}_{rwp_text(rwp)}.csv")
        
        # Load metrics from CSV file
        loaded_metrics = pd.read_csv(metrics_filename, index_col=0)
        
        # Load outliers from CSV file
        loaded_outliers_df = pd.read_csv(outliers_filename, index_col=0)
        
        return [dists_df, mea, loaded_metrics, loaded_outliers_df]
    return[dists_df, mea, None, None]

In [19]:
def analyse_site(f_train,f_test, robust, directory):
    # 4 harmonization
    harmonization_hc = harmonize(f_train, f_test, os.path.join(directory, "hc"), "No", False, True)
    harmonization_no_robust = harmonize(f_train, f_test, os.path.join(directory, "NoRobust"), "No", False, False)
    harmonization_robust = harmonize(f_train, f_test, os.path.join(directory, "robust"), robust, False, False)
    harmonization_robust_rwp = harmonize(f_train, f_test, os.path.join(directory, "robust_rwp"), robust, True, False)


    create_presentation(directory)

    #dists_analyze = compare_distances(directory, get_site(f_train), harmonization_hc[0], harmonization_no_robust[0], harmonization_robust[0], harmonization_robust_rwp[0])
    # Combine distances in a single DataFrame
    distances_combined = pd.concat([harmonization_hc[0], harmonization_no_robust[0], harmonization_robust[0], harmonization_robust_rwp[0]], ignore_index=True)
    distances_combined['method'] = ['hc', 'no_robust', 'robust', 'robust_rwp']

    # Combine MEA in a single DataFrame
    mea_combined = pd.concat([harmonization_hc[1], harmonization_no_robust[1], harmonization_robust[1], harmonization_robust_rwp[1]], ignore_index=True)
    mea_combined['method'] = ['hc', 'no_robust', 'robust', 'robust_rwp']


    #TODO bundles et analyze outliers
    return distances_combined, mea_combined, harmonization_robust[2], harmonization_robust[3]

In [ ]:
    
#Analyse Method
def analyse_method(sample_sizes, disease_ratios, num_tests, robust_method, SYNTHETIC_SITES_VERSION):
    # Split the data into training and testing sets
    directory = os.path.join(MAINFOLDER, robust_method)
    directory_site = os.path.join(SYNTHETIC_SITES ,SYNTHETIC_SITES_VERSION)
    # Initialize DataFrames to store the results
    metrics_compilation = pd.DataFrame()
    dists_compilation = pd.DataFrame()
    mea_compilation = pd.DataFrame()
    outliers_compilation = pd.DataFrame()
    for sample_size in sample_sizes:
        for disease_ratio in disease_ratios:        
            sizeDir = os.path.join(directory, f"{sample_size}_{int(disease_ratio*100)}")
            sizeDir_site = os.path.join(directory_site, f"{sample_size}_{int(disease_ratio*100)}")
            for i in range(num_tests):
                tempDir = os.path.join(sizeDir, f"{i}")
                tempDir_site = os.path.join(sizeDir_site, f"{i}")
                os.makedirs(tempDir, exist_ok=True)

                train_file_name = f"train_{sample_size}_{int(disease_ratio*100)}_{i}.csv"
                test_file_name = f"test_{sample_size}_{int(disease_ratio*100)}_{i}.csv"
                
                # Sauvegarder l'échantillon dans un fichier temporaire
                temp_file = os.path.join(tempDir_site,train_file_name )
                train_df = pd.read_csv(temp_file)
                train_df.to_csv(os.path.join(tempDir,train_file_name ), index=False)

                test_file = os.path.join(tempDir_site, test_file_name)
                test_df = pd.read_csv(test_file)
                test_df.to_csv(os.path.join(tempDir,test_file_name ), index=False)

                
                # Analyser le site pour le nouvel échantillon
                dists_analyze, mea_analyze, metrics, outliers = analyse_site(temp_file, test_file, robust_method, tempDir)
                metrics_compilation = pd.concat([metrics_compilation, metrics])
                dists_compilation = pd.concat([dists_compilation, dists_analyze])
                mea_compilation = pd.concat([mea_compilation, mea_analyze])
                outliers_compilation = pd.concat([outliers_compilation, outliers])
    # Save the metrics and distances compilation DataFrames to CSV files
    metrics_compilation.to_csv(os.path.join(directory, "metrics_compilation.csv"), index=False)
    dists_compilation.to_csv(os.path.join(directory, "dists_compilation.csv"), index=False)
    mea_compilation.to_csv(os.path.join(directory, "mea_compilation.csv"), index=False)
    outliers_compilation.to_csv(os.path.join(directory, "outliers_compilation.csv"), index=False)

## EXECUTOR

In [21]:
site_group = 'ADNI'
robust_method = 'IQR'
metric = "md"
method= "classic"

SYNTHETIC_SITES_VERSION = "v1"


sample_sizes = [30, 50, 100, 150, 200,300]  # Différentes tailles d'échantillon
disease_ratios = [0.1, 0.3, 0.5, 0.7]  # Différents pourcentages de malades
#sample_sizes = [150, 300]  # Différentes tailles d'échantillon
#disease_ratios = [0.3]  # Différents pourcentages de malades
num_tests = 10  # Nombre de tests à effectuer pour chaque combinaison

#generate_sites(sample_sizes, disease_ratios, num_tests, SYNTHETIC_SITES_VERSION)

#analyse_method(sample_sizes, disease_ratios, num_tests, robust_method, SYNTHETIC_SITES_VERSION)

## ANALYSYS

In [22]:
# Moyenne par site
# Nothing really interesting so far
directory = os.path.join(MAINFOLDER, robust_method)
dists_compilation = pd.read_csv(os.path.join(directory, "dists_compilation.csv"))
metrics_compilation = pd.read_csv(os.path.join(directory, "metrics_compilation.csv"))
directory = os.path.join(directory, ANALYSISFOLDER)
os.makedirs(directory, exist_ok=True)

dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# Display the means by site
dists_means_by_site = dists_compilation.groupby(['site','method']).mean().reset_index()
metrics_means_by_site = metrics_compilation.groupby(['site', 'metric']).mean().reset_index()

metrics_means_by_site.to_csv(os.path.join(directory, "metrics_compilation_mean.csv"), index=False)
dists_means_by_site.to_csv(os.path.join(directory, "dists_compilation_mean.csv"), index=False)
print("FINI")

FINI


In [ ]:

# Exemple d'utilisation
precision_df = calculate_precision_by_bundle(pd.read_csv(os.path.join(MAINFOLDER, robust_method, "metrics_compilation.csv")))
precision_df = precision_df.sort_values(by='precision', ascending=False)
precision_df.to_csv(os.path.join(directory, "metrics_per_bundle.csv"), index=False)

In [ ]:
precision_df = precision_df.sort_values(by='precision', ascending=False)
precision_df.head()

In [ ]:
precision_df = precision_df.sort_values(by='f1_score', ascending=False)
precision_df.head()

In [ ]:
# CREATION BOX PLOT POUR DISTANCES
def plot_bundle(df, prct, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "DISTANCES_PLOTS", str(prct))
    df = df[df['disease_ratio'] == prct *100]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','method','num_patients','disease_ratio','num_diseased']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site', 'method','num_patients','disease_ratio','num_diseased']].copy()
        methods = ["hc", "no_robust", "robust", "robust_rwp"]
        colors = ['blue', 'green', 'red', 'purple']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['num_patients'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['num_patients'] == patients][bundle_column].values 
                    for patients in bundle_df['num_patients'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Nombre de patients')
        ax.set_ylabel('Valeurs')
        ax.set_title(f'Boxplots pour le bundle: {bundle_column} avec {prct * 100}% de malades')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['num_patients'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=4, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()

# Exemple d'utilisation
distances_df = pd.read_csv(os.path.join(MAINFOLDER, robust_method, "dists_compilation.csv"))
add_nb_patients_and_diseased(distances_df)
disease_ratios = [0.1, 0.3, 0.5, 0.7]
for disease_ratio in disease_ratios:
    plot_bundle(distances_df, disease_ratio, os.path.join(MAINFOLDER, robust_method, ANALYSISFOLDER))

In [ ]:
# CREATION BOX PLOT POUR MEA
def plot_bundle(df, prct, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "MEA_PLOTS", str(prct))
    df = df[df['disease_ratio'] == prct *100]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','method','num_patients','disease_ratio','num_diseased']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site', 'method','num_patients','disease_ratio','num_diseased']].copy()
        methods = ["hc", "no_robust", "robust", "robust_rwp"]
        colors = ['blue', 'green', 'red', 'purple']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['num_patients'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['num_patients'] == patients][bundle_column].values 
                    for patients in bundle_df['num_patients'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Nombre de patients')
        ax.set_ylabel('Valeurs')
        ax.set_title(f'Boxplots pour le bundle: {bundle_column} avec {prct * 100}% de malades')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['num_patients'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=4, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()

# Exemple d'utilisation
mea_df = pd.read_csv(os.path.join(MAINFOLDER, robust_method, "mea_compilation.csv"))
add_nb_patients_and_diseased(mea_df)
disease_ratios = [0.1, 0.3, 0.5, 0.7]
for disease_ratio in disease_ratios:
    plot_bundle(mea_df, disease_ratio, os.path.join(MAINFOLDER, robust_method, ANALYSISFOLDER))

## TESTS

In [ ]:
# # # TEST ADD BIAIS
# # Split the data into training and testing sets
# directory = os.path.join(MAINFOLDER, "testBiais")
# os.makedirs(directory, exist_ok=True)
# train_df, test_df = split_train_test(CAMCAN, test_size=0.2, random_state=42)

# # Generate biased data
# # Save the original non-biased data to temporary files
# temp_train_file_original = os.path.join(directory, "temp_train_original.csv")
# temp_test_file_original = os.path.join(directory, "temp_test_original.csv")
# train_df.to_csv(temp_train_file_original, index=False)
# test_df.to_csv(temp_test_file_original, index=False)

# # Generate biased data
# sampled_df_biaied, test_df_biaised, gammas,deltas, ruffles= generate_biaised_data(train_df, test_df)

# # Save the biased data to temporary files
# temp_train_file = os.path.join(directory, "temp_train_biased.csv")
# temp_test_file = os.path.join(directory, "temp_test_biased.csv")
# sampled_df_biaied.to_csv(temp_train_file, index=False)
# test_df_biaised.to_csv(temp_test_file, index=False)

# # Run the combat_visualize_data script
# outname_train = os.path.join("visualize_train")
# cmd = (
#     "scripts/combat_visualize_data.py"
#     + " "
#     + temp_train_file_original
#     + " "
#     + temp_train_file
#     + " --out_dir "
#     + directory
#     + " --outname "
#     + outname_train
#     + " -f"
#     + " --bundles all"
# )
# subprocess.call(cmd, shell=True)

# # Display gammas and deltas along with their mean and standard deviation
# print("Gammas:", gammas)
# print("Deltas:", deltas)
# gammas = list(gammas.values())
# deltas = list(deltas.values())
# print("\nGamma Statistics:")
# print(f"Mean: {np.mean(gammas)}, Std: {np.std(gammas)}")

# print("\nDelta Statistics:")
# print(f"Mean: {np.mean(deltas)}, Std: {np.std(deltas)}")
# print("Ruffles:", ruffles)


In [ ]:
# # TEST Powerpoint generation
# d  = os.path.join(MAINFOLDER, robust_method, "adni_100_Philips_3T")
# create_presentation(d)

In [ ]:
# # TEST the sample_patients function with compilation data data
# sampled_df = sample_patients(COMPILATION, num_patients=100, disease_ratio=0.5)
# print(sampled_df)


In [ ]:
# load_metrics("ROBUST/IQR/50_30/0/", "50_patients_30_percent_0")

In [ ]:
# # Load the dists_compilation and metrics_compilation CSV files
# dists_compilation_path = os.path.join(directory, "dists_compilation.csv")
# metrics_compilation_path = os.path.join(directory, "metrics_compilation.csv")

# dists_compilation = pd.read_csv(dists_compilation_path)
# metrics_compilation = pd.read_csv(metrics_compilation_path)

# # Change the site column
# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','comparaison']).mean()
# metrics_means_by_site = metrics_compilation.groupby('site').mean()

# print(dists_means_by_site)
# print(metrics_means_by_site)

In [ ]:
# # FIX METRICS COMPILATION
# directory = os.path.join(MAINFOLDER, robust_method)
# df = pd.read_csv(os.path.join(directory, "metrics_compilation.csv"))

# # Group by the site
# grouped = df.groupby('site')

# # Process each site
# cleaned_dfs = []
# for site, group in grouped:
#     # Reset index for easier manipulation
#     group = group.reset_index(drop=True)
    
#     # # The first row is the "bundle row" (new column names)
#     # new_columns = group.iloc[0].values  # Extract column names from the first row
#     # new_columns[-1] = 'site'
#     # group = group.iloc[1:]  # Remove the first row
    
#     # # Assign new column names
#     # group.columns = new_columns
    
#     # # Sort the columns alphabetically (excluding 'site')
#     # sorted = group.sort_index(axis=1)
#     # Add a new column 'nomm' with the value indicating the metric for each row
#     metrics = ['tp', 'fp', 'tn', 'fn', 'precision', 'recall', 'taux_faux_positifs', 'f1_score']
#     group['metric'] = metrics
    
#     # # Append the cleaned DataFrame for this site
#     cleaned_dfs.append(group)

# # Concatenate all cleaned DataFrames
# final_df = pd.concat(cleaned_dfs, ignore_index=True)

# # Save or display the result
# final_df.to_csv(os.path.join(directory, "metrics_compilation.csv"), index=False)


In [ ]:
# REAL SITES
# directory = os.path.join(MAINFOLDER, robust_method)
# raw_directory = os.path.join(RAWFOLDER, site_group)
# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         analyse_site(f, robust_method, directory)
        
